tile selenium chrome windows?



In [ ]:
var importer = require('../Core');

const rows = 3;
const columns = 3;
var screen;

var openUrl = (url) => {
    var id = Math.random().toString(36).substring(7);
    return client.execute((url, id, width, height) => {
            var a = document.createElement('a');
            a.setAttribute('onclick', 'window.open("' + url + '", "", "width=' + width + ',height=' + height + '")');
            a.setAttribute('id', id);
            a.href = url;
            a.style.position = 'absolute';
            a.style.zIndex = 4294967295;
            a.style.top = 0;
            a.style.left = 0;
            a.style.bottom = 0;
            a.style.right = 0;
            document.body.appendChild(a);
        }, url, id, Math.floor(screen.width / columns), Math.floor(screen.height / rows))
        .click('a[id="' + id + '"]')
}

var createNewWindows = (count, url) => {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .then(() => {
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    const newPosition = {
                        x: Math.floor(i % columns) * Math.floor(screen.width / columns),
                        y: Math.floor(i / columns % rows) * Math.floor(screen.height / rows)
                    };
                    console.log(newPosition);
                    return openUrl(url)
                             .windowHandlePosition(newPosition)
                             .then(() => resolve())
                             .catch(e => { console.log(e); resolve() })
                }).apply(this, [i + 1]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var tileWindows = () => {
    return client
        .windowHandleMaximize()
        .windowHandleSize()
        .then(r => screen = r.value)
        .windowHandles()
        .then(h => {
            const keep = h.value.pop();
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                promises.push((i => resolve => client
                              .then(() => client.window(h.value[i]))
                              .then(() => client.close())
                              .then(() => resolve())
                              .catch(e => resolve())).apply(this, [i]));
            }
            return importer.runAllPromises(promises)
                .then(() => client.window(keep));
        })
        .then(() => client
            .windowHandlePosition({x: 0, y: 0})
            .windowHandleSize({
                width: Math.floor(screen.width / columns),
                height: Math.floor(screen.height / rows)
            }))
        .windowHandles()
        .then(h => createNewWindows(9 - h.value.length, 'https://google.com'))
        .pause(1000)
        .windowHandles()
        .catch(e => console.log(e))
}
module.exports = tileWindows;



In [17]:
var importer = require('../Core');

$$.async();
importer.import('selenium cell')
    .then(runSeleniumCell => runSeleniumCell('tile chrome windows'))
    .then(tileWindows => tileWindows())
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



Initializing webdriver on localhost
{ x: 453, y: 0 }
{ x: 906, y: 0 }
{ x: 0, y: 324 }
{ x: 453, y: 324 }
{ x: 906, y: 324 }
{ x: 0, y: 648 }
{ x: 453, y: 648 }
{ x: 906, y: 648 }


{ state: 'success',
  sessionId: '7755a460-0458-42c8-a779-061137e7d58f',
  hCode: 2121094882,
  value: 
   [ 'CDwindow-a71cbd86-c03f-43d0-9cb8-79c5e1a4bb0c',
     'CDwindow-daf56263-d910-4263-90ac-1a091f383ed3',
     'CDwindow-fe5a15a3-1111-4483-90db-dc00aa88cbef',
     'CDwindow-ec2a17ad-753d-4841-bd83-45dea6769f38',
     'CDwindow-8aa2cde3-7686-448f-9037-7bd0c7c31dd2',
     'CDwindow-a6a3f712-7a69-4a03-b2dc-a36aef452f5e',
     'CDwindow-ee13f686-3e1b-4093-a276-f5ad33a84811',
     'CDwindow-7ebd3790-3e4a-4392-91f8-a6c64e409f58',
     'CDwindow-74d3d94d-b6c5-436b-bac8-d69c522ac89a' ],
  class: 'org.openqa.selenium.remote.Response',
  status: 0 }